<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Suppose we wish to test the null hypothesis that the median of a population
is equal to 0.5. We choose a confidence level of 99%; that is, we will
reject the null hypothesis in favor of the alternative if the p-value is
less than 0.01.

When testing random variates from the standard uniform distribution, which
has a median of 0.5, we expect the data to be consistent with the null
hypothesis most of the time.


In [ ]:
import numpy as np
from scipy import stats
rng = np.random.default_rng()
rvs = stats.uniform.rvs(size=100, random_state=rng)
stats.quantile_test(rvs, q=0.5, p=0.5)

QuantileTestResult(statistic=45, statistic_type=1, pvalue=0.36820161732669576)

As expected, the p-value is not below our threshold of 0.01, so
we cannot reject the null hypothesis.

When testing data from the standard *normal* distribution, which has a
median of 0, we would expect the null hypothesis to be rejected.


In [ ]:
rvs = stats.norm.rvs(size=100, random_state=rng)
stats.quantile_test(rvs, q=0.5, p=0.5)

QuantileTestResult(statistic=67, statistic_type=2, pvalue=0.0008737198369123724)

Indeed, the p-value is lower than our threshold of 0.01, so we reject the
null hypothesis in favor of the default "two-sided" alternative: the median
of the population is *not* equal to 0.5.

However, suppose we were to test the null hypothesis against the
one-sided alternative that the median of the population is *greater* than
0.5. Since the median of the standard normal is less than 0.5, we would not
expect the null hypothesis to be rejected.


In [ ]:
stats.quantile_test(rvs, q=0.5, p=0.5, alternative='greater')

QuantileTestResult(statistic=67, statistic_type=1, pvalue=0.9997956114162866)

Unsurprisingly, with a p-value greater than our threshold, we would not
reject the null hypothesis in favor of the chosen alternative.

The quantile test can be used for any quantile, not only the median. For
example, we can test whether the third quartile of the distribution
underlying the sample is greater than 0.6.


In [ ]:
rvs = stats.uniform.rvs(size=100, random_state=rng)
stats.quantile_test(rvs, q=0.6, p=0.75, alternative='greater')

QuantileTestResult(statistic=64, statistic_type=1, pvalue=0.00940696592998271)

The p-value is lower than the threshold. We reject the null hypothesis in
favor of the alternative: the third quartile of the distribution underlying
our sample is greater than 0.6.

`quantile_test` can also compute confidence intervals for any quantile.


In [ ]:
rvs = stats.norm.rvs(size=100, random_state=rng)
res = stats.quantile_test(rvs, q=0.6, p=0.75)
ci = res.confidence_interval(confidence_level=0.95)
ci

ConfidenceInterval(low=0.284491604437432, high=0.8912531024914844)

When testing a one-sided alternative, the confidence interval contains
all observations such that if passed as `q`, the p-value of the
test would be greater than 0.05, and therefore the null hypothesis
would not be rejected. For example:


In [ ]:
rvs.sort()
q, p, alpha = 0.6, 0.75, 0.95
res = stats.quantile_test(rvs, q=q, p=p, alternative='less')
ci = res.confidence_interval(confidence_level=alpha)
for x in rvs[rvs <= ci.high]:
    res = stats.quantile_test(rvs, q=x, p=p, alternative='less')
    assert res.pvalue > 1-alpha
for x in rvs[rvs > ci.high]:
    res = stats.quantile_test(rvs, q=x, p=p, alternative='less')
    assert res.pvalue < 1-alpha

Also, if a 95% confidence interval is repeatedly generated for random
samples, the confidence interval will contain the true quantile value in
approximately 95% of replications.


In [ ]:
dist = stats.rayleigh() # our "unknown" distribution
p = 0.2
true_stat = dist.ppf(p) # the true value of the statistic
n_trials = 1000
quantile_ci_contains_true_stat = 0
for i in range(n_trials):
    data = dist.rvs(size=100, random_state=rng)
    res = stats.quantile_test(data, p=p)
    ci = res.confidence_interval(0.95)
    if ci[0] < true_stat < ci[1]:
        quantile_ci_contains_true_stat += 1
quantile_ci_contains_true_stat >= 950

True

This works with any distribution and any quantile, as long as the samples
are i.i.d.